# $SPHINCS^+$ Python : Example #

### Importing $SPHINCS^+$ Class ###

In [1]:
from package.sphincs import Sphincs

### Instantiate a $SPHINCS^+$ Object and Setting parameters ###

By default, $SPHINCS^+$ parameters are set like:
   - Security Parameter: $n=16$
   - Winternitz Parameter: $w=16$ (Should be set as 4, 16 or 256)
   - Hypertree Height: $h=64$
   - Hypertree Layers: $d=8$ ($d|h$)
   - $FORS$ Trees Number: $k=10$
   - $FORS$ Trees Height: $a=15$

Changes must be done before keys generations, signing and verification

In [2]:
sphincs = Sphincs()

sphincs.set_winternitz(4)
# Or
sphincs.set_w(4)

sphincs.set_hypertree_height(32)

### Generating a Key Pair ### 

In [3]:
sk, pk = sphincs.generate_key_pair()
print("Secret Key: ", sk)
print()
print("Public Key: ", pk)

Secret Key:  b'\x18\xf4\x9c\x12l\x03\xa2mok\xf7\x8f\xf4\xe108\xcf\xf1\xf3\xb3\x91n$\xaf\x1bLO\xd5e\r\x90\xd3dm2\x1a\x89&\xf3\xf8\xf0\xb2R\xd9\xbaUOC\x08\xc1\xdcO\x03\xd6Bp\xe0\xec\x83hDNv\x16\xd2y\x9ahXD\xee2b\xa9\xa3v\x9c\x04\x08kI\x97\x88\xd92\xb1T&f\x13K\x0fRh\xcf+2\xc6\xef\xd0\xfc.Y\x83\x96\xcal\xaea\x8b\x91_\xae\xcb-\xa6By\xf1\x8a\x86\x1c\x84AHU\xbc\xba'

Public Key:  b'\xd2y\x9ahXD\xee2b\xa9\xa3v\x9c\x04\x08kI\x97\x88\xd92\xb1T&f\x13K\x0fRh\xcf+2\xc6\xef\xd0\xfc.Y\x83\x96\xcal\xaea\x8b\x91_\xae\xcb-\xa6By\xf1\x8a\x86\x1c\x84AHU\xbc\xba'


### Signing a Message ###

The message must be in a $bytes()$ format

In [4]:
m = b"No one knows the reason for all this, but it is probably quantum. - Pyramids, Terry Pratchett (1989)"
print("Message to be signed: ", m)

Message to be signed:  b'No one knows the reason for all this, but it is probably quantum. - Pyramids, Terry Pratchett (1989)'


In [5]:
signature = sphincs.sign(m, sk)

In [6]:
signature

b'uI\xdd\x90v\t\x07P\xabE\x82\xa8\xc9\x83\x1b\xeb\xc0\x05\t\xe0\xfa\xbey\xc4l\x88?\x1a\xbb\xe9\x88\xfc\x90*L\xea\xea\xfc/\x06}\xfe\x91bj\xd8\xa4\x8d\t\xac\x13i\xee9`)\x94\xd2\xbbq\xd7\xb1\xee\t\xa2\x86V./"\x82\x17Fr\xe2\x87\x9a8\xba\xc6\xa2\xecI\x04\x1frm\x85\xe3\x18lG\xc0,\x83\x08\x85"#\x95B\xee2/j\x05xm\xb8\x9d#\xa1B\xad\x16\x1f\x13\xaf2\x9a\xc9\xeb:\xe3\x87\xae\x81\xb2\x9c\x0b\xab\xdb\xb5\xa51\x8bQ8\x86T\xf6\xd8\x01\xed`i\x1d\xeb\xd3M\x11pP\xf1\xc5\xbc\xad\xa4\x94\xb5\x01y(\x03\x13\x96\x08\xc6S\x106\xf8\xb7O;\xc7\xd3\x9a\xfd\xb0\\b\xfb\xf8R-`\x88n\xce\xba\xa7\xbc\x8a\xe0\xfd\xfe\xde,\xf0\x04\xcb\x1d\xce\x18\xf7-\xbc\xfb\xfc\x7f\x7f\xae\x86\xdd\x93\xff\x822\x13\xc2\xddR\x92M$\xc1+\x0e\xeaqxo\xb8E\x82e\\\\\xf5\x07\x1a\xc3o\n\x98[\xa6\xdaH\x8f\x0e*&S\xf6\xac\xc0\xf1\r\xdf\xcey<\xb5\xcdy\x06\x9e\x01\xc0\xff@\xf8:\\[\xaf\x88\xefd\x01\xc0hi"\x1e\xaf\xd5\x86\xd7\xcf\xe8#8\xe3\xc1\xbb\x11\x9f4\x80\xf8(\x80\'\xce8\x12\xfc\xba\xaa]\xb6\x1b\xe4X\x0c\x19\x96\xdeo\xe1Bf+B8\x8e\x92\x8c\xabb+\xe1\

In [7]:
print("Signature bytes-size: ", len(signature))

Signature bytes-size:  7684


### Verifying a Signature associated with a Message ###

In [8]:
print("Is signature correct ? ", sphincs.verify(m, signature, pk))

Is signature correct ?  True
